In [1]:
pip install emoji --upgrade


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.9/240.9 KB 5.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for emoji: filename=emoji-2.2.0-py3-none-any.whl size=234926 sha256=4d32b7d945eb6e6b407d32b42ba27af0b763605c97c2e45747fa61a2ec3a3a79
  Stored in directory: /root/.cache/pip/wheels/86/62/9e/a6b27a681abcde69970dbc0326ff51955f3beac72f15696984
Successfully built emoji


In [2]:
import emoji
import numpy as np

In [3]:
#emoji.EMOJI_UNICODE
#this shows a dictionary in which each emoji is matched to some keywords

In [4]:
# we will use only 5 emojis(0-4) because our training data has examples of only these emojies
emoji_dictionary={'0':'\u2764\uFE0F', #heart prints a  heart
                 '1':':baseball:',
                 '2':':grinning_face_with_big_eyes:',
                 '3':':disappointed_face:',
                 '4':':fork_and_knife:',
                 '5':':hundred_points:',
                  '6':':fire:',
                  '7':':face_blowing_a_kiss:',
                  '8':':chestnut:',
                  '9':':flexed_biceps:'
                 }

In [5]:
#emojize function is used to print the emojis using given keyword related to it(if present)
for e in emoji_dictionary.values():
    print(emoji.emojize(e))

❤️
⚾
😃
😞
🍴
💯
🔥
😘
🌰
💪


In [ ]:
#Load The training and testing data
import pandas as pd
train=pd.read_csv('train_emoji.csv',header=None)
test=pd.read_csv('test_emoji.csv',header=None)

In [ ]:
test.head()
#labels have been given to the emojis

,0,1
0,I want to eat\t,4
1,he did not answer\t,3
2,he got a raise\t,2
3,she got me a present\t,0
4,ha ha ha it was so funny\t,2


In [ ]:
xtrain=train[0]
ytrain=train[1]
#first column is sentences and second column is labels

In [ ]:
#We try to print the sentences and relevant emojis
for i in range(ytrain.shape[0]):
    print(xtrain[i],end=' ')
    print(emoji.emojize(emoji_dictionary[str(ytrain[i])]))

never talk to me again 😞
I am proud of your achievements 😃
It is the worst day in my life 😞
Miss you so much ❤️
food is life 🍴
I love you mum ❤️
Stop saying bullshit 😞
congratulations on your acceptance 😃
The assignment is too long  😞
I want to go play ⚾
she did not answer my text  😞
Your stupidity has no limit 😞
how many points did he score ⚾
my algorithm performs poorly 😞
I got approved 😃
Stop shouting at me 😞
Sounds like a fun plan ha ha 😃
no one likes him 😞
the game just finished ⚾
I will celebrate soon 😃
So sad you are not coming 😞
She is my dearest love ❤️
Good job 😃
It was funny lol 😃
candy is life  😃
The chicago cubs won again ⚾
I am hungry 🍴
I am so excited to see you after so long 😃
you did well on you exam 😃
lets brunch some day 🍴
he is so cute ❤️
How dare you ask that 😞
do you want to join me for dinner  🍴
I said yes 😃
she is attractive ❤️
you suck 😞
she smiles a lot 😃
he is laughing 😃
she takes forever to get ready  😞
French macaroon is so tasty 🍴
we made it 😃
I am excited

In [ ]:
xtest=test[0]
ytest=test[1]

In [ ]:
#We use a pre trained embedding layer i.e Glove Vector
#it has 6 billions words and 50 sized vector for each word
f=open('glove.6B.50d.txt',encoding='utf-8')
#from this text file we create our won dictionary of words and their corresponding vectors

In [ ]:
embeddings_index={}
#initialise and empty dict.
for line in f:
    values=line.split()
    #divide the lines over space
    #first word is the word reqd and rest is its vector
    word=values[0]
    coeff=np.asarray(values[1:],dtype='float')
    #print(word,coeff)
    embeddings_index[word]=coeff

In [ ]:
#we have got vectors corresponding to each word
#now we will get vectors corresponding to sentences
def embedding_output(X):
    #fix max num of words in a sentence =10
    maxlen=10
    emb_dim=50
    #dimesnion of one vector
    embedding_out=np.zeros((X.shape[0],maxlen,emb_dim))
    #embedding_out[i][j] represents vector of jth word in ith line
    for i in range(X.shape[0]):
        words=X[i].split()
        #extract the words from each line
        for j in range(len(words)):
            try :
                #then assign its corresponding vector
                embedding_out[i][j]=embeddings_index[words[j].lower()]
            except:
                #incase the word is not present in our dict.
                embedding_out[i][j]=np.zeros((50,))
    return embedding_out

In [ ]:
embedding_matrix_train=embedding_output(xtrain)
embedding_matrix_test=embedding_output(xtest)
#convert the training and testing data to embedded layer o/p format

In [ ]:
from keras.models import Sequential
from keras.layers import LSTM,Dropout,Dense

In [ ]:
#initialise the model
model=Sequential()
#size of activation vector is kept 64
#return sequences=true means that o/p of this LSTM will be fed to the next LSTM Layer
model.add(LSTM(64,input_shape=(10,50),return_sequences=True))
#Dropout Layer
model.add(Dropout(0.5))
#Since we have only 2 LSTM Layer so last layer will give the output to the classification layer(i.e dense layer)
model.add(LSTM(64,return_sequences=False))
model.add(Dropout(0.5))
#no of class=5
model.add(Dense(5,activation='softmax'))

In [ ]:
#loss is categorical cross entropy as it is a multi class classification
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [ ]:
from keras.utils import to_categorical
#model need to be fed in one hot notation
ytrain=to_categorical(ytrain,num_classes=5)
ytest=to_categorical(ytest,num_classes=5)

In [ ]:
from keras.callbacks import EarlyStopping,ModelCheckpoint
#use callbacks like checkpoint and earlystopping for best val acuracy
checkpoint=ModelCheckpoint("best_model.h5",monitor='val_loss',verbose=True,save_best_only=True)
earlystop=EarlyStopping(monitor='val_loss',patience=10)
hist=model.fit(embedding_matrix_train,ytrain,epochs=100,batch_size=64,shuffle=True,validation_split=0.2,callbacks=[checkpoint,earlystop])

Epoch 1/100
2/2 [==============================] - 4s 770ms/step - loss: 1.5848 - accuracy: 0.2421 - val_loss: 1.6167 - val_accuracy: 0.2222

Epoch 00001: val_loss improved from inf to 1.61667, saving model to best_model.h5
Epoch 2/100
2/2 [==============================] - 0s 43ms/step - loss: 1.5535 - accuracy: 0.3010 - val_loss: 1.6225 - val_accuracy: 0.2222

Epoch 00002: val_loss did not improve from 1.61667
Epoch 3/100
2/2 [==============================] - 0s 45ms/step - loss: 1.5238 - accuracy: 0.3506 - val_loss: 1.6337 - val_accuracy: 0.1852

Epoch 00003: val_loss did not improve from 1.61667
Epoch 4/100
2/2 [==============================] - 0s 43ms/step - loss: 1.5041 - accuracy: 0.3622 - val_loss: 1.6485 - val_accuracy: 0.2593

Epoch 00004: val_loss did not improve from 1.61667
Epoch 5/100
2/2 [==============================] - 0s 45ms/step - loss: 1.4840 - accuracy: 0.3801 - val_loss: 1.6581 - val_accuracy: 0.2222

Epoch 00005: val_loss did not improve from 1.61667
Epoch 6/

In [ ]:
preds=model.predict_classes(embedding_matrix_test)
#predict the values

C:\Users\DEVIL_Pc\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.8_qbz5n2kfra8p0\LocalCache\local-packages\Python38\site-packages\tensorflow\python\keras\engine\sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


In [ ]:
model.evaluate(embedding_matrix_test,ytest)
#check the accuracy

2/2 [==============================] - 0s 5ms/step - loss: 1.2614 - accuracy: 0.6071


[1.2613991498947144, 0.6071428656578064]

In [ ]:
for i in range(30):
    print(xtest[i],emoji.emojize(emoji_dictionary[str(preds[i])]))

I want to eat	 🍴
he did not answer	 😞
he got a raise	 ⚾
she got me a present	 ❤️
ha ha ha it was so funny	 😃
he is a good friend	 😃
I am upset	 ⚾
We had such a lovely dinner tonight	 😃
where is the food	 🍴
Stop making this joke ha ha ha	 😃
where is the ball	 ⚾
work is hard	 😃
This girl is messing with me	 ❤️
are you serious ha ha	 😞
Let us go play baseball	 ⚾
This stupid grader is not working 	 😞
work is horrible	 😃
Congratulation for having a baby	 😃
stop messing around	 😞
any suggestions for dinner	 😃
I love taking breaks	 ❤️
you brighten my day	 ❤️
I boiled rice	 🍴
she is a bully	 ❤️
Why are you feeling bad	 😞
I am upset	 ⚾
I worked during my birthday	 😃
My grandmother is the love of my life	 ❤️
enjoy your break	 ❤️
valentine day is near	 😃
